_This is a simple program to demonstrate a teething issue we are having with the correct structuring of the primitive set._

1. First we set up the deap program environment, then
1. declare a primitive set that works and run it, then
1. redefine that primitive set to remove the terminals and re-run it, thus exposing the error.

In [1]:
%load_ext autoreload

In [2]:
import random
import numpy
import math

from deap import gp
from deap import creator
from deap import base
from deap import tools
from deap import algorithms

In [3]:
PLANE_SIZE = 8
POPULATION_SIZE = 100


class Point(object):
    """
    A point on a plane
    """
    def __init__(self, x, y):
        """
        x and y is a black point on a white photo
        """
        self.x = x
        self.y = y

    def __repr__(self):
        return "<%s, %s>" % (self.x, self.y)

    def getx(self):
        return self.x

    def gety(self):
        return self.y

    @staticmethod
    def distance(this, that):
        """Return the distance between the point on this and the point provided"""
        return math.sqrt((this.x - that.x) ** 2 + (this.y - that.y) ** 2)


class RandomPoint(Point):
    def __init__(self):
        super(RandomPoint, self).__init__(*[random.randrange(0, PLANE_SIZE) for _ in range(2)])


A_RANDOMPOINTS = [RandomPoint() for _ in range(10)]


# A smaller score (distance) is better
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

# Every individual is a tree of operations (plus a Fitness class)
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)


def get_toolbox(pset):
    """
    :return: a configured toolbox
    """
    toolbox = base.Toolbox()
    toolbox.register("expr", gp.genGrow, pset, min_=1, max_=50)
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("compile", gp.compile, pset=pset)

    def eval_func(individual):
        """
        Apply the program to pairs of triangles in the sample set.  Tally up the differences
        between the output and the goal.
        """
        # print(individual)
        program = toolbox.compile(expr=individual)
        score = 0
        for point_in in A_RANDOMPOINTS:
            point_out = program(point_in)
            diff = Point.distance(point_in, point_out)
            score += diff
        return score,

    toolbox.register("evaluate", eval_func)
    toolbox.register("select", tools.selBest)
    toolbox.register("mate", gp.cxOnePoint)
    toolbox.register("expr_mut", gp.genGrow, min_=0, max_=5)
    toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
    return toolbox


def run(toolbox):
    pop = toolbox.population(n=POPULATION_SIZE)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("max", numpy.max)
    stats.register("min", numpy.min)
    return algorithms.eaSimple(pop, toolbox, 0.5, 0.2, 10, stats, halloffame=hof)


def main():
    pop, logbook = run(get_toolbox(get_pset()))
    for i in sorted(pop, key=lambda i: i.fitness.values[0], reverse=True):
        print(int(i.fitness.values[0]), i)

### This primitive set works...

In [4]:
def get_pset():
    """
    :return: a typed set of primitive operations
    """
    pset = gp.PrimitiveSetTyped(
        "DAIN",
        [Point],
        Point
    )

    # get coordinates of a point
    pset.addPrimitive(Point.getx, [Point], int, name='getx')
    pset.addPrimitive(Point.gety, [Point], int, name='gety')

    # create a point
    pset.addPrimitive(Point, [int, int], Point, name='new_point')

    # create a terminal for every int
    pset.addEphemeralConstant("Rint", lambda: random.randint(0, PLANE_SIZE), int)

    # give the input args more meaningful names
    pset.renameArguments(ARG0='in_point')
    return pset

In [5]:
main()

gen	nevals	avg    	std    	max   	min
0  	100   	36.6923	13.2213	66.832	0  
1  	52    	34.4631	15.0736	71.6567	0  
2  	54    	34.0232	16.0816	71.6567	0  
3  	63    	34.2702	16.1582	69.0133	0  
4  	62    	35.3787	15.4741	69.0133	0  
5  	63    	36.3848	15.6943	69.0133	0  
6  	60    	36.4968	15.1506	69.0133	0  
7  	53    	36.9276	15.1722	69.0133	0  
8  	66    	36.8312	15.2251	69.0133	0  
9  	59    	35.656 	15.5763	69.0133	0  
10 	71    	35.598 	15.2367	69.0133	0  
69 new_point(8, 8)
69 new_point(getx(new_point(8, 8)), 8)
64 new_point(7, gety(new_point(gety(new_point(6, gety(new_point(gety(new_point(2, gety(in_point))), gety(new_point(7, getx(in_point))))))), getx(new_point(gety(new_point(4, getx(new_point(8, getx(in_point))))), 5)))))
61 new_point(8, getx(new_point(7, getx(in_point))))
61 new_point(getx(new_point(8, getx(new_point(gety(new_point(gety(new_point(5, 8)), getx(in_point))), gety(new_point(8, 2)))))), 7)
59 new_point(1, 7)
59 new_point(gety(in_point), 8)
57 new_point(0, gety(ne

### Now the same primitive set but without the availability of the random integer (a Terminal):

In [8]:
def get_pset():
    """
    :return: a typed set of primitive operations
    """
    pset = gp.PrimitiveSetTyped(
        "DAIN",
        [Point],
        Point
    )

    # get coordinates of a point
    pset.addPrimitive(Point.getx, [Point], int, name='getx')
    pset.addPrimitive(Point.gety, [Point], int, name='gety')

    # create a point
    pset.addPrimitive(Point, [int, int], Point, name='new_point')

    # REMOVING THIS: create a terminal for every int
    # pset.addEphemeralConstant("Rint", lambda: random.randint(0, PLANE_SIZE), int)

    # give the input args more meaningful names
    pset.renameArguments(ARG0='in_point')
    return pset

### This one doesn't work:

Rerunning without the random ints creates this problem where it is not prepared to use the `getx` or `gety` (integer generating functions) to complete the tree.  I now believe this is because these functions would make the tree 2 layers deeper, whereas the `gengrow` function has been told to grow the tree only one more layer.  

When it says it has _nothing available_, it means **nothing of the required type and depth of nodes**.  In this case it wants an integer which needs just one layer.  But we took those away, the random int terminals, so we get this error:

```The gp.generate function tried to add a terminal of type '<class 'int'>', but there is none available.```

In [9]:
main()

IndexError: The gp.generate function tried to add a terminal of type '<class 'int'>', but there is none available.